In [3]:
from Vissim_env_class import env

%matplotlib inline

In [4]:
model_name  = 'Balance'
vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
sim_length = 3601

# all controller actions
Controllers_Actions =\
{\
    # Controller Number 2 
    0 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 3
    1 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 4
    2 : {
            0 : [1,0,0,0,1,0,0,0],
            1 : [0,1,0,0,0,1,0,0],
            2 : [0,0,1,0,0,0,1,0],
            3 : [0,0,0,1,0,0,0,1]
        },
    # Controller Number 5
    3 : {
            0 : [1,0,0,1,0,0],
            1 : [0,1,0,0,1,0],
            2 : [0,0,1,0,0,1],
        },
    # Controller Number 6 
    4 : {
            0 : [1,1,0,0],
            1 : [0,1,1,0],
            2 : [1,0,0,1]
        },
    # Controller Number 8
    5 : {
            0 : [1,0,1],
            1 : [0,1,0]
        },
    # Controller Number 9
    6 : {
            0 : [1,0,1,0],
            1 : [0,1,0,1]
        },
    # Contoller Number 10
    7 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,1,0],
            2 : [0,0,1,0,1]
        },
    # Controller Number 12
    8 : {
            0 : [1,0,1],
            1 : [0,1,0],
        },
    # Controller Number 13
    9 : {
            0 : [1,0,0,0],
            1 : [0,1,0,1],
            1 : [0,0,1,0],
        },
    # Controller 15
     10 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 16
    11 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 17
    12 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,0,1],
            2 : [0,0,1,0,0],
            3 : [0,0,0,1,1],
            4 : [0,1,0,0,1]
        },
    # Controller 33 
    13 : {
            0 : [1,0,0,1],
            1 : [1,0,1,0],
            2 : [0,1,0,0]
        }
}

In [5]:
env = env(model_name, vissim_working_directory, sim_length, Controllers_Actions,\
            timesteps_per_second = 1, mode = 'training', delete_results = True, verbose = True)

Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Balance.inpx ...
Load process successful
Simulation length set to 3601 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************

0.4326171875
0.40071964263916016
0.2868194580078125
0.2955207824707031
0.5391364097595215
0.5479106903076172
0.39926934242248535
0.4269390106201172
0.16583585739135742
0.14960741996765137
0.18452858924865723
0.17149686813354492
0.21844053268

In [6]:
env.SCUs

{0: <Vissim_SCU_class.Signal_Control_Unit at 0x176900e54a8>,
 1: <Vissim_SCU_class.Signal_Control_Unit at 0x17690137f98>,
 2: <Vissim_SCU_class.Signal_Control_Unit at 0x17690137d68>,
 3: <Vissim_SCU_class.Signal_Control_Unit at 0x17690137cf8>,
 4: <Vissim_SCU_class.Signal_Control_Unit at 0x17690137f28>,
 5: <Vissim_SCU_class.Signal_Control_Unit at 0x176901379b0>,
 6: <Vissim_SCU_class.Signal_Control_Unit at 0x17690137e48>,
 7: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a2b0>,
 8: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a208>,
 9: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a1d0>,
 10: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a6a0>,
 11: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a198>,
 12: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a6d8>,
 13: <Vissim_SCU_class.Signal_Control_Unit at 0x1769014a4a8>}

In [7]:
env.SCUs[0].state

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [8]:
env.reset()

0.3998994827270508
0.39499473571777344
0.3038668632507324
0.30283665657043457
0.5345809459686279
0.528083324432373
0.41228699684143066
0.4001505374908447
0.16585588455200195
0.16936564445495605
0.18842625617980957
0.1889793872833252
0.2074742317199707
0.22158050537109375
0.36039280891418457
0.37743687629699707
0.25957632064819336
0.2541522979736328
0.28036069869995117
0.45362019538879395
0.43219780921936035
0.45748186111450195
0.38405656814575195
0.30368852615356445
0.4236757755279541
0.43534278869628906
0.29247403144836426
0.2820117473602295
